# Information Extraction
We sometimes need to extract data from documents.
For example, most documents contain dates and names of people or places.
We can collect this information for example for statistics or further processing.


(regex)=
## Regular Expressions
Regular expressions (regexes) are powerful tools for searching text.
We have already seen different ways to search for text.
If we want to find the word "Article" in the string `document`, we can use one of the following methods:
- `if 'Article' in document:`
- `position = document.find('Article')`
- `document.startswith('Article')` (to check only the first word)

With regular expressions, we can look for patterns, not just specific strings.
To use regular expressions, we must import Python's `re` module:

In [ ]:
import re

We will need some text to search, so we will use the first page of a recent ECtHR case as an example.

In [ ]:
document = '''NORWEGIAN CONFEDERATION OF TRADE UNIONS (LO) AND NORWEGIAN
TRANSPORT WORKERS’ UNION (NTF) v. NORWAY JUDGMENT

In the case of Norwegian Confederation of Trade Unions (LO) and
Norwegian Transport Workers’ Union (NTF) v. Norway,
The European Court of Human Rights (Fifth Section), sitting as a
Chamber composed of:
Síofra O’Leary, President,
Mārtiņš Mits,
Stéphanie Mourou-Vikström,
Lətif Hüseynov,
Jovan Ilievski,
Ivana Jelić, judges,
Anne Grøstad, ad hoc judge,
and Victor Soloveytchik, Section Registrar,
Having regard to:
the application against the Kingdom of Norway lodged with the Court
under Article 34 of the Convention for the Protection of Human Rights and
Fundamental Freedoms (“the Convention”) by two Norwegian associations,
the Norwegian Confederation of Trade Unions (Landsorganisasjonen i
Norge (“LO”)) and the Norwegian Transport Workers’ Union (Norsk
transportarbeiderforbund (“NTF”)) (“the applicant unions”), on 15 June
2017;
the withdrawal of Arnfinn Bårdsen, the judge elected in respect of
Norway, from sitting in the case (Rule 28 § 3 of the Rules of Court) and the
decision of the President of the Section to appoint Anne Grøstad to sit as an
ad hoc judge (Article 26 § 4 of the Convention and Rule 29 § 1(a));
the decision to give notice to the Norwegian Government (“the
Government”) of the complaint concerning Article 11 of the Convention
and to declare inadmissible the remainder of the application;
the observations submitted by the respondent Government and the
observations in reply submitted by the applicants;
the comments submitted by the European Trade Union Confederation
(ETUC), which was granted leave to intervene by the President of the
Section;
Having deliberated in private on 18 May 2021,
Delivers the following judgment, which was adopted on that date:

INTRODUCTION

1. The case concerns the alleged violation of Article 11 of the
Convention in relation to a decision by the Norwegian Supreme Court to
declare unlawful an announced boycott by a trade union which was planned
in order to pressure a Norwegian subsidiary of a Danish company to enter
into a Norwegian collective agreement applicable to dockworkers.'''

We can look for the word "Article" followed by any digit.
`\d` matches a single digit:

In [ ]:
matches = re.findall(r'Article \d', document)
print(matches)

::::::{admonition} Raw Strings
:class: tip
Backslashes have special meaning in Python strings.
They are used to make special characters like tab, `\t`, and newline, `\n`, called *escape sequences*.
To get an actual backslash, we must escape it with another backslash: 
```
'Article \\d'
```

We can avoid this by using *raw* strings.
They are prefixed with an "r", like f-strings are prefixed with an "f":
```
r'Article \d'
```
This is especially helpful for patterns with many backslashes.
::::::

### Quantifiers

We can repeat characters or patterns with *quantifiers*:
- `?` matches 0 or 1 instance of the preceding item.
The expression `Article ?5` *only* matches the two strings "Article5" and "Article 5".
- `*` matches 0 or more instances of the preceding item.
For example, the expression `Article *5` matches "Article5", "Article 5" and so on, with any amount of space between the word and the digit.
- `+` matches 1 or more instances of the preceding item.
The expression `Article +5` matches "Article 5" and all variants with at least one space before the 5.

To match numbers with multiple digits, we use the expression `\d+`.

In [ ]:
print(re.findall(r'Article \d+', document))

### Character Classes

Our document mentions both Articles and Rules, so we can expand our expression to match both.
This can be done in different ways.
First, let's look at the different *character classes* we can match.
- \w matches a single *word* character. This includes the letters a to z, but also letters from other languages and alphabets.
- \s matches a single *space* character, including space, tab, and others.
- \d matches a single digit, as we have seen.
- . (period) matches any single character. 

You can also make your own character class, by listing the characters in brackets.
For example, [abc] matches a single a, b or c.

Now, let's try to match any word followed by any number.

In [ ]:
print(re.findall(r'\w+ \d+', document))

### Or
As we can see, this also matches other substrings such as "on 15".
Instead, we should specify just the words we want to look for.
We can do that with a `|`, which is used as *or* in regular expressions.

In [ ]:
print(re.findall(r'Article|Rule \d+', document))

### Grouping
Now, the digits are interpreted as belonging with "Rule" only, because of the *precedence* rules of regular expressions. We need to group the words with parentheses: `(?: | )`.

In [ ]:
print(re.findall(r'(?:Article|Rule) \d+', document))

We can also include the paragraph number:

In [ ]:
print(re.findall(r'(?:Article|Rule) \d+ § \d+', document))

However, now we only find occurrences that include a paragraph number.
We can make that part optional by putting it in parentheses followed by `?`:

In [ ]:
print(re.findall(r'(?:Article|Rule) \d+(?: § \d+)?', document))

### Ignoring Case

Our current regular expression only matches strings containing "Article" or "Rule" exactly,
with uppercase initials and the rest of the letters in lowercase.
We might also want to ignore the case of the letters,
so that we also match "article" and "rule" written in other case variants.
To do that we include the *flag* `re.IGNORECASE`.

In [ ]:
print(re.findall(r'(?:Article|Rule) \d+(?: § \d+)?', document, flags=re.IGNORECASE))

### Anchors
Earlier, we have seen the string search methods `.startswith()` and `.endswith()` that only looks for matches at the beginning or end of strings.
We can do this with regular expressions by using anchors.

For example, the tabular ECHR-OD data has several column names that contains the word "article".
The columns with names `article=*` give the relevant articles for each case.
On the other hand, the columns with names `ccl_article=*` have the conclusion for each article.

We want to filter the columns, selecting only the ones giving the relevant articles,
i.e. starting with "article".
We can try matching on `article-\d`:

In [ ]:
print(re.findall(r'article-\d', '''article-5
                             ccl_article-6'''))

This also matches "article-6", which is incorrect.
To fix this, we can use a `^` anchor:

In [ ]:
print(re.findall(r'^article-\d', '''article-5
                             ccl_article-6'''))

This expression correctly matches only lines starting with "article".
The table {ref}`regex-anchors` lists the anchors we can use in regular expressions.

:::{table} Regex Anchors
:name: regex-anchors
| Expression | Matches             |
|------------|---------------------|
| ^          | Beginning of string |
| $          | End of string       |
| \b         | Word boundaries     |

:::

### Further Reading

Regular expressions have many more features, and you can read more about them in for example
[Python's Regular Expression HOWTO](https://docs.python.org/3/howto/regex.html#regex-howto)
or the [Wikipedia article Regular expression](https://en.wikipedia.org/wiki/Regular_expression).

## Named Entity Recognition
We can use Named Entity Recognition (NER) to extract names of people, companies, places and other entities.
Most NER systems also extract numbers.
We will use the NER module in [spaCy]( https://spacy.io/).
spaCy is a library for Natural Language Processing in Python.

First, we import the spaCy library and its module displaCy.

In [ ]:
import spacy
from spacy import displacy

spaCy can use models for many different languages.
The first time we use NER in spaCy we must download the data files for the English language.

In [ ]:
!python -m spacy download en_core_web_sm

We load a short text document to run the Named Entity Recognizer on.

In [ ]:
filename = 'LO-NTF-v-Norway.txt'
with open(filename, 'r', encoding='utf-8') as file:
    text = file.read()

We load the English NLP model:

In [ ]:
nlp = spacy.load("en_core_web_sm")

Next, we process the text with the NLP model.

In [ ]:
document = nlp(text)

We can extract the entities and their labels:

In [ ]:
entities = [(ent.text, ent.label_) for ent in document.ents]

Let's look at the data:

In [ ]:
for entity in entities:
    print(f"Entity: {entity[0]}, Label: {entity[1]}")

We can also get the entity types:

In [ ]:
identity_types = set(ent.label_ for ent in document.ents)
print(f"Identity types: {identity_types}")

Finally, we can display the tagged text.

In [ ]:
# Visualize text with named entities as tags
displacy.render(document, style="ent", jupyter=True)